In [1]:
%load_ext autoreload
%autoreload 3

import fv_mapz
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from importlib import reload 

#---------------------------------------------------------------------
def write_Z_profile(data, data_keys = ['OLD_PE', 'NEW_PE', 'OLD_DZ', 'NEW_DZ', 'OLD_W', 'NEW_W']):
    
    from columnar import columnar
        
    newlist = []
    newlist.append(np.arange(data['NEW_PE'].shape[0]).tolist())
    
    headers = ['level']
    
    for key in data_keys:
        
        newlist.append(data[key][:].tolist())
        headers.append(key)

    # need to rearange all the data into row like an excel spreadsheet
    
    row_data = [list(x) for x in zip(*newlist)]
        
    table = columnar(row_data, headers, no_borders=True)
    print(table)


In [2]:
%%file sim_w_1d.f90

!>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
subroutine SIM_W_1D(dt, km, rgas, gama, gm2, cp2, kappa, pe, dm2, pm2, pem, w2, dz2, pt2, ws, p_fac)
                        
   integer, intent(in)  :: km
   real,    intent(in)  :: dt, rgas, gama, kappa, p_fac
   real,    intent(in), dimension(km):: dm2, pt2, pm2, gm2, cp2
   real,    intent(in ) ::  ws
   real,    intent(in ), dimension(km+1):: pem
   real,    intent(inout) ::  pe(km+1), dz2(km), w2(km)

! Local variables

   real, dimension(km  ):: Ak, Bk, Rk, Ck, g_rat, bb, dd, aa, cc, dz1, w1
   real, dimension(km+1):: pp, gam, wE, dm2e, wES
   real  rdt, capa1, bet

   rdt   = 1. / dt
   capa1 = kappa - 1.

! Compute non-hydrostatic pert pressure

   do k = 1,km
    
     pe(k)   = exp(gama*log(-dm2(k)/dz1(k)*rgas*pt2(k))) - pm2(k)        
     dm2e(k) = dm2(k)
    
     w1(k)   = w2(k)
     dz1(k)  = dz2(k)
        
   enddo

   dm2e(km+1) = dm2(km)
    
! Set up tridiagonal coeffs for spline interpolation of w to grid edges. (Extra copy of bb for end calcs)

    do k = 1,km-1
       
      g_rat(k) = dm2e(k)/dm2e(k+1)
    
      aa(k)    = 1.0
      bb(k)    = 2.*(1.+g_rat(k))
      cc(k)    = g_rat(k)
      dd(k)    = 3.*(w1(k) + cc(k)*w1(k+1))

    enddo
    
! Boundary conditions for wE = 0 at top

    bet    = bb(1)
    wE(1)  = 0.0
    wE(2)  = dd(1) / bet
    bb(km) = 2.
    dd(km) = 3.*w1(km)
       
! Forward calculation of tri-diagonal system

    do k=2,km
    
      gam(k)  =  g_rat(k-1) / bet
      bet     =  bb(k) - gam(k)
      wE(k+1) = (dd(k) - wE(k) ) / bet
         
    enddo

! Boundary conditions for wE = [ws(i) at ground]  
  
      wE(km+1) = 0.0

! Do the back substition, result is wE on zone edges.

    do k = km, 2, -1

      wE(k) = wE(k) - gam(k)*wE(k+1)
        
    enddo
    
! Validated the wE here.

   wES(:) = wE(:)
    
! Compute cell centered tridiagonal coefficients 

    do k = 1, km
    
      aa(k) = dt * gama * (pm2(k)+pe(k)) / dz1(k) 
        
    enddo

! Compute edge centered tridiagonal coefficients and RHS

    do k = 2, km

      cc(k) = dt / (0.5 * (dm2(k) + dm2(k-1))) 
        
      Bk(k) = 1.0 + cc(k) * (aa(k-1) - aa(k))
    
      Ak(k) = aa(k-1)*cc(k)
        
      Ck(k) = aa(k  )*cc(k)
    
      Rk(k) = wE(k) + cc(k) * (pe(k) - pe(k-1))

    enddo    

! Boundary value calc for forward tri-diagonal solution

   Rk(2)  = Rk(2)  - 0.0 * Ck(2)    ! this includes the lower bc for w zero here
    
   Rk(km) = Rk(km) - 0.0 * Ck(km)   ! this includes the lower bc for w zero here
  
! Forward sweep.

    bet    = Bk(2)

    wE(1)  = 0.0
    
    wE(2)  = Rk(2) / bet
    
    do k = 3, km

      gam(k)  = Ck(k-1) / bet
        
      bet     = Bk(k) - Ak(k) * gam(k)

      wE(k)   = (Rk(k) - wE(k-1) ) / bet

    enddo
    
    wE(km+1) = 0.0
            
! Back substitution for solution (wE = 0 at k=1)

    do k = km-1, 2, -1
      
      wE(k) = wE(k) - gam(k+1)*wE(k+1)
            
    enddo
        
! Solve for new perturbation pressure

    do k = 1, km

      pp(k)  = pe(k) - aa(k) * (wE(k+1) - wE(k))

    enddo
    
    
! Use new pp at cell centers to get new dz's

    do k = 1, km
    
      dz2(k) = -dm2(k)*rgas*pt2(k)*exp(capa1*log(pp(k)+pm2(k)))

    enddo
    
! Need to generate new w at cell centers...use the time tendency of dz

    do k = 1,km

     w2(k) = w1(k) + rdt * (dz2(k) - dz1(k))
          
    enddo
    
! Retrieve edge pressures

   pe(1) = 0.0

    do k = 1,km
      
      pe(k+1) = pe(k) + dm2(k)*(w2(k)-w1(k))*rdt
        
    enddo
    
 end subroutine SIM_W_1D


Overwriting sim_w_1d.f90


In [3]:
!fcompile.py



   ---> Removing all module files...safety first!


rm: cannot remove '*.mod': No such file or directory

Fortran object files compiled:   



rm: cannot remove 'sim_w_1d.so': No such file or directory


 ==> f2py --debug --fcompiler=gnu95 --f90flags='-O1 -funroll-loops -fopenmp -fPIC -I/usr/local/include' -DF2PY_REPORT_ON_ARRAY_COPY -c -m sim_w_1d sim_w_1d.f90  -lgomp -L/usr/local/lib  


running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "sim_w_1d" sources
f2py options: []
f2py:> /tmp/tmpopzo0u8w/src.linux-x86_64-3.9/sim_w_1dmodule.c
creating /tmp/tmpopzo0u8w/src.linux-x86_64-3.9
Reading fortran codes...
	Reading file 'sim_w_1d.f90' (format:free)
Post-processing...
	Block: sim_w_1d
			Block: sim_w_1d
Post-processing (stage 2)...
Building modules...
	Building module "sim_

In [5]:
# read text filea into pandas DataFrame and label columns
#
df1 = pd.read_fwf("fort/fort.1",header=None)

df2 = pd.read_fwf("fort/fort.2", header=None)

df3 = pd.read_fwf("fort/fort.3", header=None)

df_pE = pd.read_fwf("fort/fort.4", header=None)

dfPre = pd.concat([df1, df2, df3], axis=1)
dfPre.columns = ['k', 'pe', 'pm2', 'w2', 'dz2', 'dm2', 'pt2', 'gm2', 'cp2']

before = {}

nz = 60
nrec = len(dfPre['k'])
ntime = int(nrec/nz)
print(ntime)

for key in ['k', 'pe', 'pm2', 'w2', 'dz2', 'dm2', 'pt2', 'cp2', 'gm2']:
    
    before[key] = dfPre[key].to_numpy().reshape(ntime,nz)

before['pem'] = df_pE[0].to_numpy().reshape(ntime, nz+1)


df1 = pd.read_fwf("fort/fort.97",header=None)

df2 = pd.read_fwf("fort/fort.98", header=None)

dfPost = pd.concat([df1, df2], axis=1)
dfPost.columns = ['k', 'pe', 'pm2', 'w2', 'dz2', 'dm2']

after = {}


for key in ['k', 'pe', 'pm2', 'w2', 'dz2', 'dm2']:
    
    after[key] = dfPost[key].to_numpy().reshape(ntime,nz)
    
########################################################################

time = 300

data = {}
    
data['NEW_PE'], data['NEW_DZ'],data['NEW_W'] = after['pe'][time], after['dz2'][time], after['w2'][time]

data['OLD_PE'], data['OLD_DZ'], data['OLD_W'] = before['pe'][time], before['dz2'][time], before['w2'][time]

write_Z_profile(data)
    

324
                
  LEVEL  OLD_PE              NEW_PE               OLD_DZ             NEW_DZ             OLD_W                NEW_W                
    
  0      -0.250283792886876  0.00016620003598027  -3200.70424453486  -3200.59048710329  -0.0137969335837426  -0.0141571227375812  
                             17                                                         3                    3                    
  1      -0.351674974919661  -0.751244590150056   -1967.53441358023  -1967.6128195396   -0.108667991854622   -0.108599416326115   
  2      0.176866641233573   0.0596073282914981   -1531.21101347034  -1531.22508204981  -0.422288676973688   -0.421888063804911   
  3      0.24927081802889    0.117409229272347    -1094.97527362574  -1094.98471372776  -0.492103143921219   -0.491557320209657   
  4      0.92995108166906    1.07558861399214     -834.899789640644  -834.892832366914  -0.470692158714212   -0.470072213914486   
  5      1.39271809760976    1.24539325569802     -684.93

In [12]:
import sim_w_1d

sim_w_1d = reload(sim_w_1d)


print(sim_w_1d.__doc__)

dt    =  0.166666666666667        
rgas  = 287.050000000000        
gama  = 1.40004180893318
kappa = 0.285735616165638       
p_fac = 0.000000000000000
ws    = 0.0

time = 300

pe = np.zeros((nz+1,))
pe[1:] = before['pe'][time,:]

print(pe)

#This module 'sim_w_1d' is auto-generated with f2py (version:1.20.3).
#Functions:
# sim_w_1d(dt, rgas, gama, gm2, cp2, kappa, dm2, pm2, pem, w1, dz1, pt2, ws, p_fac, km=len(gm2))

data = {}

data['OLD_PE'], data['OLD_DZ'], data['OLD_W'] = before['pe'][time], before['dz2'][time], before['w2'][time]


sim_w_1d.sim_w_1d(dt, rgas, gama, before['gm2'][time,:], before['cp2'][time,:], kappa, pe, \
                                                         before['dm2'][time,:], before['pm2'][time,:], before['pem'][time,:], \
                                                                 before['w2'][time,:],  before['dz2'][time,:], before['pt2'][time,:], ws, p_fac)

write_Z_profile(data,data_keys = ['OLD_PE', 'NEW_PE', 'NEW_DZ', 'OLD_W', 'NEW_W'] )

This module 'sim_w_1d' is auto-generated with f2py (version:1.20.3).
Functions:
  sim_w_1d(dt,rgas,gama,gm2,cp2,kappa,pe,dm2,pm2,pem,w2,dz2,pt2,ws,p_fac,km=len(gm2))
.
[ 0.00000000e+00 -2.50283793e-01 -3.51674975e-01  1.76866641e-01
  2.49270818e-01  9.29951082e-01  1.39271810e+00  1.91336322e+00
  2.33908381e+00  2.82453549e+00  3.24798813e+00  3.74952702e+00
  4.19365844e+00  4.72144716e+00  5.18442458e+00  5.73739810e+00
  6.22166303e+00  6.78963918e+00  7.27520364e+00  7.84295264e+00
  8.31492730e+00  8.85950351e+00  9.28402557e+00  9.75137863e+00
  1.00402436e+01  1.02778161e+01  1.01445488e+01  9.60033911e+00
  7.85654122e+00  5.75596248e+00  6.78341738e+00  7.72101438e+00
  7.75625371e+00  7.12269339e+00  5.30609126e+00  1.90615660e+00
 -4.50260681e+00 -1.60232529e+01 -3.76244176e+01 -6.57898283e+01
 -7.33385127e+01 -4.89411757e+01 -4.67408634e+00  5.28682459e+01
  1.12192010e+02  1.68476083e+02  2.20084143e+02  2.64036349e+02
  2.99022207e+02  3.25943836e+02  3.45807705e+02  3.

ValueError: failed in converting 7th argument `pe' of sim_w_1d.sim_w_1d to C/Fortran array